# Time Series Data Processing

This notebook processes scraped time series data by:
1. Loading the raw scraped data
2. Cleaning and standardizing date formats
3. Handling missing values
4. Creating time-series visualizations
5. Exporting cleaned data to CSV

## 1. Import Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime
import warnings

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Configure matplotlib for better plots
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Libraries loaded successfully!")

## 2. Load Scraped Data

Load the raw data from `scraped_data.csv` that was created by `scrape.py`.

In [ ]:
# Load the scraped data
INPUT_FILE = 'scraped_data.csv'
OUTPUT_FILE = 'cleaned_timeseries.csv'

try:
    df_raw = pd.read_csv(INPUT_FILE)
    print(f"Loaded {len(df_raw)} rows from {INPUT_FILE}")
    print(f"\nColumns: {list(df_raw.columns)}")
except FileNotFoundError:
    print(f"Error: {INPUT_FILE} not found.")
    print("Please run 'python scrape.py' first to fetch the data.")
    raise

In [ ]:
# Display first few rows to understand the data structure
print("Raw Data Preview:")
print("=" * 50)
df_raw.head(10)

In [ ]:
# Check data types and missing values
print("Data Types:")
print("-" * 30)
print(df_raw.dtypes)
print("\nMissing Values:")
print("-" * 30)
print(df_raw.isnull().sum())

## 3. Clean Date Formats

The raw data may contain dates in various formats. We'll standardize them to a consistent datetime format.

In [ ]:
def parse_date(date_str):
    """
    Parse dates in multiple formats and return a standardized datetime.
    
    Supported formats:
    - 2024-01-02 (ISO format)
    - 01/02/2024 (US format)
    - 01-02-2024 (Dash format)
    - 2024/01/02 (Slash ISO)
    - Jan 02, 2024 (Month name)
    """
    date_formats = [
        '%Y-%m-%d',      # 2024-01-02
        '%m/%d/%Y',      # 01/02/2024
        '%m-%d-%Y',      # 01-02-2024
        '%Y/%m/%d',      # 2024/01/02
        '%b %d, %Y',     # Jan 02, 2024
        '%B %d, %Y',     # January 02, 2024
        '%d-%m-%Y',      # 02-01-2024 (European)
        '%d/%m/%Y',      # 02/01/2024 (European)
    ]
    
    if pd.isna(date_str):
        return pd.NaT
    
    date_str = str(date_str).strip()
    
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    
    # Try pandas parser as fallback
    try:
        return pd.to_datetime(date_str)
    except:
        return pd.NaT

print("Date parsing function defined.")

In [ ]:
# Create a copy for cleaning
df = df_raw.copy()

# Parse dates using our custom function
print("Parsing dates...")
print("\nSample of original dates:")
print(df['Date'].head(10).tolist())

df['Date'] = df['Date'].apply(parse_date)

print("\nSample of parsed dates:")
print(df['Date'].head(10).tolist())

# Check for any parsing failures
failed_parses = df['Date'].isna().sum()
if failed_parses > 0:
    print(f"\nWarning: {failed_parses} dates could not be parsed")
else:
    print("\nAll dates parsed successfully!")

## 4. Clean Numeric Columns

Ensure all price and volume columns are properly typed as numeric values.

In [ ]:
# Define numeric columns
numeric_cols = ['Open', 'High', 'Low', 'Close', 'Volume']

# Convert to numeric, coercing errors to NaN
for col in numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        print(f"{col}: converted to numeric")

print("\nData types after conversion:")
print(df.dtypes)

In [ ]:
# Handle missing values
print("Missing values before cleaning:")
print(df.isnull().sum())

# Drop rows with missing dates (critical field)
initial_rows = len(df)
df = df.dropna(subset=['Date'])
dropped_dates = initial_rows - len(df)

# Forward fill missing numeric values (common for time series)
for col in numeric_cols:
    if col in df.columns:
        df[col] = df[col].fillna(method='ffill').fillna(method='bfill')

print(f"\nDropped {dropped_dates} rows with missing dates")
print("\nMissing values after cleaning:")
print(df.isnull().sum())

## 5. Sort and Validate Data

In [ ]:
# Sort by date
df = df.sort_values('Date').reset_index(drop=True)

# Basic validation
print("Data Validation:")
print("=" * 50)
print(f"Date range: {df['Date'].min()} to {df['Date'].max()}")
print(f"Total trading days: {len(df)}")
print(f"\nPrice Statistics:")
print(df[['Open', 'High', 'Low', 'Close']].describe())

In [ ]:
# Verify data integrity (High >= Low, etc.)
integrity_issues = []

if 'High' in df.columns and 'Low' in df.columns:
    invalid_hl = (df['High'] < df['Low']).sum()
    if invalid_hl > 0:
        integrity_issues.append(f"{invalid_hl} rows where High < Low")

if 'High' in df.columns and 'Close' in df.columns:
    invalid_hc = (df['High'] < df['Close']).sum()
    if invalid_hc > 0:
        integrity_issues.append(f"{invalid_hc} rows where High < Close")

if 'Low' in df.columns and 'Close' in df.columns:
    invalid_lc = (df['Low'] > df['Close']).sum()
    if invalid_lc > 0:
        integrity_issues.append(f"{invalid_lc} rows where Low > Close")

if integrity_issues:
    print("Data Integrity Issues Found:")
    for issue in integrity_issues:
        print(f"  - {issue}")
else:
    print("Data integrity check passed!")

## 6. Time Series Visualization

Create visualizations to understand trends in the data.

In [ ]:
# Create figure with multiple subplots
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Plot 1: Closing Price Over Time
ax1 = axes[0]
ax1.plot(df['Date'], df['Close'], color='#2196F3', linewidth=1.5, label='Close Price')
ax1.fill_between(df['Date'], df['Close'], alpha=0.3, color='#2196F3')
ax1.set_title('Closing Price Over Time', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date')
ax1.set_ylabel('Price ($)')
ax1.legend(loc='upper left')
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax1.xaxis.set_major_locator(mdates.AutoDateLocator())
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Plot 2: OHLC Range (High-Low Spread)
ax2 = axes[1]
ax2.fill_between(df['Date'], df['Low'], df['High'], alpha=0.4, color='#4CAF50', label='High-Low Range')
ax2.plot(df['Date'], df['Open'], color='#FF9800', linewidth=1, linestyle='--', label='Open', alpha=0.8)
ax2.plot(df['Date'], df['Close'], color='#2196F3', linewidth=1, label='Close')
ax2.set_title('Price Range (Open, High, Low, Close)', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Price ($)')
ax2.legend(loc='upper left')
ax2.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax2.xaxis.set_major_locator(mdates.AutoDateLocator())
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Plot 3: Trading Volume
ax3 = axes[2]
ax3.bar(df['Date'], df['Volume'], color='#9C27B0', alpha=0.7, width=0.8)
ax3.set_title('Trading Volume Over Time', fontsize=14, fontweight='bold')
ax3.set_xlabel('Date')
ax3.set_ylabel('Volume')
ax3.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax3.xaxis.set_major_locator(mdates.AutoDateLocator())
plt.setp(ax3.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Format y-axis for volume (millions)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x/1e6:.1f}M'))

plt.tight_layout()
plt.savefig('timeseries_visualization.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nVisualization saved as 'timeseries_visualization.png'")

In [ ]:
# Additional analysis: Daily returns
df['Daily_Return'] = df['Close'].pct_change() * 100

fig, ax = plt.subplots(figsize=(14, 5))

colors = ['#4CAF50' if x >= 0 else '#F44336' for x in df['Daily_Return'].fillna(0)]
ax.bar(df['Date'], df['Daily_Return'], color=colors, alpha=0.7, width=0.8)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax.set_title('Daily Returns (%)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Return (%)')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
ax.xaxis.set_major_locator(mdates.AutoDateLocator())
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

# Print return statistics
print("\nDaily Return Statistics:")
print(f"  Mean: {df['Daily_Return'].mean():.4f}%")
print(f"  Std Dev: {df['Daily_Return'].std():.4f}%")
print(f"  Min: {df['Daily_Return'].min():.4f}%")
print(f"  Max: {df['Daily_Return'].max():.4f}%")

## 7. Export Cleaned Data

In [ ]:
# Prepare final dataframe for export
# Remove the Daily_Return column if you only want the core data
df_export = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()

# Format date as ISO string for CSV export
df_export['Date'] = df_export['Date'].dt.strftime('%Y-%m-%d')

# Round numeric values for cleaner output
df_export['Open'] = df_export['Open'].round(2)
df_export['High'] = df_export['High'].round(2)
df_export['Low'] = df_export['Low'].round(2)
df_export['Close'] = df_export['Close'].round(2)
df_export['Volume'] = df_export['Volume'].astype(int)

print("Final cleaned data preview:")
df_export.head(10)

In [ ]:
# Export to CSV
df_export.to_csv(OUTPUT_FILE, index=False)

print(f"Cleaned data exported to '{OUTPUT_FILE}'")
print(f"\nExport Summary:")
print(f"  Total rows: {len(df_export)}")
print(f"  Columns: {list(df_export.columns)}")
print(f"  Date range: {df_export['Date'].iloc[0]} to {df_export['Date'].iloc[-1]}")
print(f"  File size: {os.path.getsize(OUTPUT_FILE) / 1024:.2f} KB" if 'os' in dir() else '')

In [ ]:
# Verify the exported file
import os

df_verify = pd.read_csv(OUTPUT_FILE)
print("Verification - Reading exported file:")
print(f"  Rows: {len(df_verify)}")
print(f"  Columns: {list(df_verify.columns)}")
print("\nFirst 5 rows:")
print(df_verify.head())
print("\nLast 5 rows:")
print(df_verify.tail())

## Summary

This notebook completed the following tasks:

1. **Loaded** raw scraped data from `scraped_data.csv`
2. **Cleaned** date formats from multiple input formats to standardized ISO format
3. **Validated** numeric columns and handled missing values
4. **Created** time-series visualizations showing price trends and volume
5. **Exported** cleaned data to `cleaned_timeseries.csv`

The cleaned dataset is now ready for further analysis, modeling, or integration with other systems.